# Conversion to GeoDataFrame for Visualization with HoloViz Packages
Authors: [Philip Chmielowiec](https://github.com/philipc2), [Ian Franda](https://github.com/ifranda)

## Overview
This usage example notebook showcases how UXarray data structures (`Grid`, `UxDataArray`) can be converted in a Spatialpandas `GeoDataFrame`, which can be used directly by packages from the HoloViz stack, such as hvPlot, Datashader, Holoviews, and Geoviews. In addition to showing how to convert to a `GeoDataFrame`, a series of visualizations using hvPlot and GeoViews is showcased based around the converted data structure.

```{note}
While the visualizations in this notebook are not rendered using UXarray functionality, development of UXarray's own visualization functions can be tracked [here](https://github.com/UXARRAY/uxarray/milestone/2).
```



## Imports

This notebook requires the following visualization packages to be installed in order to run locally.

```
conda install -c pyviz holoviews hvplot geoviews 
conda install -c conda-forge spatialpandas antimeridian
```


In [1]:
import uxarray as ux
import numpy as np
import warnings

import matplotlib
import matplotlib.pyplot as plt

import hvplot.pandas
import holoviews as hv

warnings.filterwarnings("ignore")

## Data

For this notebook, we will be using E3SM model output, which uses a cubed-sphere grid. However, this notebook can be adapted to datasets in any of our supported formats.

In [2]:
base_path = "../../test/meshfiles/ugrid/outCSne30/"
grid_path = base_path + "outCSne30.ug"
data_path = base_path + "outCSne30_vortex.nc"

In [3]:
uxds = ux.open_dataset(grid_path, data_path)

In [4]:
uxds

<xarray.UxDataset>
Dimensions:  (ncol: 5400)
Dimensions without coordinates: ncol
Data variables:
    psi      (ncol) float64 1.351 1.331 1.31 1.289 ... 0.7121 0.6909 0.67 0.6495

In [5]:
uxds.uxgrid

<uxarray.Grid>
Original Grid Type: UGRID
Grid Dimensions:
  * nMesh2_face: 5400
  * nMaxMesh2_face_nodes: 4
  * nMesh2_node: 5402
Grid Coordinate Variables:
  * Mesh2_node_x: (5402,)
  * Mesh2_node_y: (5402,)
Grid Connectivity Variables:
  * Mesh2_face_nodes: (5400, 4)
  * nNodes_per_face: (5400,)

## Conversion to `spatialpandas.GeoDataFrame` for Visualization

In order to support visualization with the popular HoloViz stack of libraries (hvPlot, HoloViews, Datashader, etc.), UXarray provides methods for converting `Grid` and `UxDataArray` objects into a SpatialPandas `GeoDataFrame`, which can be used for visualizing the nodes, edges, and polygons that represent each grid, in addition to data variables.


### `Grid` Conversion

If you wish to plot *only* the grid's geometrical elements such as nodes and edges (i.e. without any data variables mapped to them), you can directly convert a `Grid` instance into a `GeoDataFrame`.

Each `UxDataset` and `UxDataArray` has a `Grid` instance paired to it, which can be accessed using the `.uxgrid` attribute.

You can use the `Grid.to_geodataframe()` method to construct a `GeoDataFrame`.

In [6]:
gdf_grid = uxds.uxgrid.to_geodataframe()
gdf_grid

,geometry
0,"MultiPolygon([[[-45.0, -35.26438968275467, -42..."
1,"MultiPolygon([[[-42.0, -36.617694956996594, -3..."
2,"MultiPolygon([[[-39.0, -37.85242110467453, -36..."
3,"MultiPolygon([[[-36.0, -38.97344733686565, -33..."
4,"MultiPolygon([[[-33.0, -39.98557075458056, -30..."
...,...
5395,"MultiPolygon([[[147.3315024327531, 43.07367919..."
5396,"MultiPolygon([[[144.19934215834053, 42.0115829..."
5397,"MultiPolygon([[[141.0996896078898, 40.83760372..."
5398,"MultiPolygon([[[138.03317101605955, 39.5490780..."


### `UxDataArray` & `UxDataset` Conversion

If you are interested in mapping data to each face, you can index the `UxDataset` with the variable of interest (in this case "psi") to return the same `GeoDataFrame` as above, but now with data mapped to each face.

```{warning}
UXarray currently only supports mapping data variables that are mapped to each face. Variables that reside on the corner nodes or edges are currently not supported for visualization.
```



In [7]:
gdf_data = uxds['psi'].to_geodataframe()
gdf_data

,geometry,psi
0,"MultiPolygon([[[-45.0, -35.26438968275467, -42...",1.351317
1,"MultiPolygon([[[-42.0, -36.617694956996594, -3...",1.330915
2,"MultiPolygon([[[-39.0, -37.85242110467453, -36...",1.310140
3,"MultiPolygon([[[-36.0, -38.97344733686565, -33...",1.289056
4,"MultiPolygon([[[-33.0, -39.98557075458056, -30...",1.267717
...,...,...
5395,"MultiPolygon([[[147.3315024327531, 43.07367919...",0.733539
5396,"MultiPolygon([[[144.19934215834053, 42.0115829...",0.712085
5397,"MultiPolygon([[[141.0996896078898, 40.83760372...",0.690883
5398,"MultiPolygon([[[138.03317101605955, 39.5490780...",0.669989


### Challenges with Representing Geoscience Data as Geometries

When we convert to a `GeoDataFrame`, we internally represent the surface of a sphere as a collection of polygons over a 2D projection. However, this leads to issues around the Antimeridian (180 degrees east or west) where polygons are incorrectly constructed and have incorrect geometries. When constructing the `GeoDataFrame`, UXarray detects and corrects any polygon that touches or crosses the antimeridian. An array of indices of these faces can be accessed as part of the `Grid` object.
<br>

<figure>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Earth_map_with_180th_meridian.jpg/640px-Earth_map_with_180th_meridian.jpg" style="height: 300px; width:600px;"/></center>
<center><figcaption>Antimeridian Visual</figcaption></center>
</figure>


In [8]:
uxds.uxgrid.antimeridian_face_indices

array([1814, 1844, 1874, 1904, 1934, 1964, 1994, 2024, 2054, 2084, 2114,
       2144, 2174, 2204, 2234, 2264, 2294, 2324, 2354, 2384, 2414, 2444,
       2474, 2504, 2534, 2564, 2594, 2624, 2654, 2684, 3615, 3645, 3675,
       3705, 3735, 3765, 3795, 3825, 3855, 3885, 3915, 3945, 3975, 4005,
       4034, 4035, 4964, 4965, 4995, 5025, 5055, 5085, 5115, 5145, 5175,
       5205, 5235, 5265, 5295, 5325, 5355, 5385])

Taking a look at one of these faces that crosses or touches the antimeridian, we can see that it's split across the antimeridian and represented as a `MultiPolygon`, which allows us to properly render this two dimensional grid.

In [9]:
gdf_data.geometry[uxds.uxgrid.antimeridian_face_indices[0]]

MultiPolygon([[[180.0, -42.0, 177.0, -41.960930170814336, 177.0, -44.96071214756905, 180.0, -45.00000000000001, 180.0, -42.0]], [[-180.0, -45.00000000000001, -180.0, -45.00000000000001, -180.0, -42.0, -180.0, -42.0, -180.0, -45.00000000000001]]])

For more details about the algorithm used for splitting these polygons, see the [Antimeridian Python Package](https://antimeridian.readthedocs.io/en/stable/), which we use internally for correcting polygons on the antimeridian.


## Visualizing Geometries

The next sections will show how the `GeoDataFrame` representing our unstructured grid can be used with hvPlot and GeoViews to generate visualizations. Examples using both the `matplotlib` and `bokeh` backends are shown, with `bokeh` examples allowing for interactive plots.




### Nodes

In [10]:
# use matplotlib backend for rendering
hv.extension("matplotlib")

plot_kwargs = {"size": 6.0, "xlabel": "Longitude", "ylabel": "Latitude", "coastline": True, "width": 1600, "title": "Node Plot (Matplotlib Backend)"}


gdf_grid.hvplot.points(**plot_kwargs)

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W


KeyboardInterrupt



In [ ]:
# use bokeh backend for rendering
hv.extension("bokeh")

plot_kwargs = {"s": 1.0, "xlabel": "Longitude", "ylabel": "Latitude", "coastline": True, "frame_width": 700, "title": "Node Plot (Bokeh Backend)"}

gdf_grid.hvplot.points(**plot_kwargs)

### Edges

In [ ]:
# use matplotlib backend for rendering
hv.extension("matplotlib")

plot_kwargs = {"linewidth": 1.0, "xlabel":" Longitude", "ylabel": "Latitude", "coastline": True, "width": 1600 , "title": "Edge Plot (Matplotlib Backend)", "color": "black"}

import cartopy.crs as ccrs

gdf_grid.hvplot.paths(**plot_kwargs)

In [ ]:
# use bokeh backend for rendering
hv.extension("bokeh")

plot_kwargs = {"line_width": 0.5, "xlabel": "Longitude", "ylabel": "Latitude", "coastline": True, "frame_width": 700, "title": "Edge Plot (Bokeh Backend)"}

gdf_grid.hvplot.paths(**plot_kwargs)

## Visualizing Data Variables

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"c": "psi", "cmap": "inferno", "width": 400, "height": 200, "title": "Filled Polygon Plot (Matplotlib Backend, Rasterized)", "xlabel":" Longitude", "ylabel": "Latitude"}

gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True)

```{note}
Visualizing filled polygons without rasterization using the matplotlib backend produces incorrect results, see [hvplot/#1099](https://github.com/holoviz/hvplot/issues/1099)
```

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"c": "psi",  "cmap": "inferno", "line_width": 0.1,  "frame_width": 500, "frame_height": 250, "xlabel":" Longitude", "ylabel": "Latitude"}

gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True)

hv.Layout(gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True).opts(title="Filled Polygon Plot (Bokeh Backend, Rasterized)") +
          gdf_data.hvplot.polygons(**plot_kwargs, rasterize=False).opts(title="Filled Polygon Plot (Bokeh Backend, Vector)")).cols(1)

## Geographic Projections



In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"c": "psi", "cmap": "inferno", "width": 400, "height": 200, "title": "Filled Polygon Plot (Matplotlib Backend, Rasterized, Orthographic Projection)", "xlabel":" Longitude", "ylabel": "Latitude"}

gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True, projection=ccrs.Orthographic())

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"c": "psi", "cmap": "inferno", "width": 400, "height": 200, "title": "Filled Polygon Plot (Matplotlib Backend, Rasterized, Robinson Projection)", "xlabel":" Longitude", "ylabel": "Latitude"}

gdf_data.hvplot.polygons(**plot_kwargs, rasterize=True, projection=ccrs.Robinson())